In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [67]:
# Create DataFrames from CSV files
t0 = time.time()
filtered_tf_idf = pd.read_csv('../../Data/filtered_tf_idf.csv')
filtered_tf_idf = filtered_tf_idf.drop([0, 1])
filtered_tf_idf.index = filtered_tf_idf.index - 2
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_tf_idf.head(3)

Run time: 66.73523592948914 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,39968,39976,39980,39982,39983,39987,39990,39991,39998,40000
0,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1913.0
1,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.662,0.413,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1439.0
2,Heat Waves,Glass Animals,pop,8.0,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,1.0,0.0944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2087.0


In [68]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = filtered_tf_idf[filtered_tf_idf['category_name']==cat]['category_id'].values[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [69]:
# Print how many songs there are in each category
for cat in category_list:
    song_count = len(filtered_tf_idf[filtered_tf_idf['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 452 songs in the blues category.
There are 110 songs in the classical category.
There are 1277 songs in the country category.
There are 308 songs in the funk category.
There are 704 songs in the hiphop category.
There are 696 songs in the indie_alt category.
There are 231 songs in the jazz category.
There are 931 songs in the metal category.
There are 709 songs in the pop category.
There are 598 songs in the punk category.
There are 368 songs in the rnb category.
There are 1257 songs in the rock category.
There are 283 songs in the romance category.
There are 420 songs in the soul category.


In [70]:
# Get the most popular genres for each category
t0 = time.time()
category_genres = {}
for category in category_list:
    genres = []
    for index, row in filtered_tf_idf[filtered_tf_idf['category_name']==category].iterrows():
        genres.extend(row['genre_list'])
    unique_genres = list(set(genres))
    genre_counts = []
    for genre in unique_genres:
        genre_counts.append((genre, genres.count(genre)))
    category_genres[category] = sorted(genre_counts, key = lambda x: x[1], reverse=True)[0:5]
    print(category, category_genres[category])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

blues [("'", 4334), (' ', 1738), (',', 1738), ('e', 1350), ('c', 1208)]
classical [("'", 694), ('a', 281), (' ', 246), (',', 246), ('i', 217)]
country [("'", 6448), ('o', 3624), ('r', 3313), ('n', 2625), ('t', 2258)]
funk [("'", 2754), (' ', 1084), (',', 1084), ('o', 860), ('n', 616)]
hiphop [("'", 5788), (' ', 2227), (',', 2227), ('a', 1685), ('r', 1643)]
indie_alt [("'", 6430), ('e', 2657), (' ', 2539), (',', 2539), ('n', 1900)]
jazz [("'", 1806), (' ', 685), (',', 685), ('o', 653), ('a', 539)]
metal [("'", 8996), (' ', 3585), (',', 3585), ('e', 3135), ('a', 2957)]
pop [("'", 3738), ('e', 1373), ('p', 1367), (' ', 1248), (',', 1248)]
punk [("'", 4290), (' ', 1582), (',', 1582), ('e', 1437), ('o', 1249)]
rnb [("'", 3700), (' ', 1489), (',', 1489), ('n', 929), ('e', 845)]
rock [("'", 11166), (' ', 4361), (',', 4361), ('e', 3535), ('r', 3064)]
romance [("'", 2506), (' ', 975), (',', 975), ('o', 721), ('e', 574)]
soul [("'", 4016), (' ', 1603), (',', 1603), ('o', 1156), ('s', 867)]
Run t

In [71]:
# Find indeces to keep and indeces to remove
t0 = time.time()
cat_genre_dict = {'pop': ['pop'], 
                  'hiphop': ['rap', 'hip', 'trap'],
                  'metal': ['metal'], 
                  'rock': ['classic'], 
                  'jazz': ['jazz'], 
                  'rnb': ['r&b'], 
                  'romance': ['soft'], 
                  'soul': ['soul'], 
                  'indie_alt': ['indie', 'alternative'],
                  'classical': ['classical', 'operatic', 'dance'], 
                  'blues': ['modern', 'blues'],
                  'funk': ['funk', 'disco'], 
                  'punk': ['punk'],
                  'country': ['country']} 
indeces_to_keep = []
indeces_to_remove = []
for index, row in filtered_tf_idf.iterrows():
    keep = False
    category = row['category_name']
    for genre in cat_genre_dict[category]:
        if genre in row['genre_list']:
            keep = True
            break
    if keep == True:
        indeces_to_keep.append(index)
    else:
        indeces_to_remove.append(index)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(indeces_to_keep))
len(indeces_to_remove)

Run time: 39.04143214225769 seconds
4329


4015

In [72]:
# Create non destructive copy and remove the unnecessary indeces
t0 = time.time()
filtered_tf_idf2 = filtered_tf_idf.drop(indeces_to_remove)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_tf_idf2.head(3)

Run time: 0.3995859622955322 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,39968,39976,39980,39982,39983,39987,39990,39991,39998,40000
0,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1913.0
3,r u ok,Tate McRae,pop,8.0,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1668.0
4,BOYSHIT,Madison Beer,pop,8.0,"['dance', 'pop', 'electropop', 'post-teen']",0.638,0.699,1.0,1.0,0.0892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1802.0


In [73]:
# Remove the romance, punk, and jazz categories
filtered_tf_idf2 = filtered_tf_idf2[filtered_tf_idf2['category_name']!='romance']
filtered_tf_idf2 = filtered_tf_idf2[filtered_tf_idf2['category_name']!='punk']
filtered_tf_idf2 = filtered_tf_idf2[filtered_tf_idf2['category_name']!='blues']
#filtered_tf_idf2 = filtered_tf_idf2[filtered_tf_idf2['category_name']!='indie_alt']
len(filtered_tf_idf2)

3726

In [74]:
# Print how many songs there are in each category 
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    song_count = len(filtered_tf_idf2[filtered_tf_idf2['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 0 songs in the blues category.
There are 46 songs in the classical category.
There are 987 songs in the country category.
There are 177 songs in the funk category.
There are 453 songs in the hiphop category.
There are 409 songs in the indie_alt category.
There are 96 songs in the jazz category.
There are 494 songs in the metal category.
There are 389 songs in the pop category.
There are 0 songs in the punk category.
There are 213 songs in the rnb category.
There are 251 songs in the rock category.
There are 0 songs in the romance category.
There are 211 songs in the soul category.


In [75]:
# Define the features set
X = filtered_tf_idf2.copy()
X = X.drop(['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list', 'lyrics', 'filtered'], axis=1)
X.head(3)

,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,...,39968,39976,39980,39982,39983,39987,39990,39991,39998,40000
0,0.581,0.584,2.0,1.0,0.2840,0.0805,0.0,0.3660,0.756,179.954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1913.0
3,0.666,0.593,2.0,1.0,0.0373,0.3180,0.0,0.4140,0.329,140.013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1668.0
4,0.638,0.699,1.0,1.0,0.0892,0.0203,0.0,0.0879,0.570,121.122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1802.0


In [76]:
# Define the target set
y = filtered_tf_idf2['category_id'].ravel()
y = y.astype('int')
y[:5]

array([8, 8, 8, 8, 8])

In [77]:
# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2794, 13115)
(932, 13115)
(2794,)
(932,)


In [78]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the Model

In [79]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [80]:
# Fit the model
t0 = time.time()
rf_model = rf_model.fit(X_train_scaled, y_train)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 50.5954430103302 seconds


# Making Predictions

In [81]:
# Make predictions using the testing data
predictions = rf_model.predict(X_test_scaled)
predictions

array([ 2,  7,  4,  4,  7,  4,  5,  2,  7,  2,  2,  7,  2,  2,  5,  7,  2,
        4,  4,  2,  2,  2, 13,  2,  2,  7,  2,  7,  7,  4,  2,  2,  2,  2,
        2,  2,  2,  7,  2,  2,  2,  2,  4,  7,  7,  2,  2,  2,  7,  7,  5,
        2,  2,  2,  2,  2,  8,  8,  2,  2,  2,  4, 11,  7,  2,  2, 13,  2,
        2,  2,  2,  2,  2,  2,  4,  2,  4,  2,  7,  2,  4,  2,  5,  4,  4,
        7,  2,  2,  2,  2,  4, 11,  2,  2,  2,  4,  4, 13,  2,  2,  2,  2,
        2,  4,  2, 10,  2,  2,  2,  4,  2,  2,  7,  2,  5,  2,  7,  2,  4,
        2,  5,  4,  7,  5,  2,  2,  4,  7,  7,  7,  3,  2,  2,  2,  7,  2,
        7,  2,  7,  2,  2,  2,  2,  2,  8,  2,  2,  2,  2,  2,  2,  2,  4,
        4,  2,  2,  2,  2,  4, 13,  2,  5,  7,  2,  4,  2,  2,  2,  4,  2,
        2,  2,  4,  2,  5,  2,  3,  2,  7,  7,  7,  5,  7,  5,  4,  5,  2,
        2,  7,  2,  2,  2,  2,  2,  2,  2,  2,  2,  5,  2,  2,  7,  2,  2,
        2,  2,  2,  2,  4,  2,  2,  4,  2,  7,  2,  2,  2,  2,  5,  5,  7,
        2,  2,  2,  4,  4

# Model Evaluation

In [82]:
# Calculate the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.5815450643776824


In [83]:
# Calculate the feature importance in the Random Forest Model
importances = rf_model.feature_importances_
importances

array([0.01442731, 0.01819554, 0.00405544, ..., 0.00010083, 0.        ,
       0.01209916])

In [84]:
# Sort the features by importance
sorted_features = sorted(zip(importances, X.columns), reverse=True)
sorted_features

[(0.021344856497173038, 'audio_ft_acousticness'),
 (0.018195537958860284, 'audio_ft_energy'),
 (0.016602215488149698, 'audio_ft_speechiness'),
 (0.01442730592751484, 'audio_ft_danceability'),
 (0.014219430108110784, 'audio_ft_instrumentalness'),
 (0.012099162826865987, '40000'),
 (0.011888026662880817, 'audio_ft_duration_ms'),
 (0.010427530295293223, 'audio_ft_valence'),
 (0.007184143049365949, '33372'),
 (0.0068345561928243245, 'audio_ft_tempo'),
 (0.0067480651567471146, 'audio_ft_liveness'),
 (0.005117665623459625, '3623'),
 (0.005060643827121858, '16240'),
 (0.0050260882629049024, 'audio_ft_mode'),
 (0.004873602381396108, '25397'),
 (0.004825003740551269, '3330'),
 (0.004810124157389658, '23249'),
 (0.004784338221740122, '21348'),
 (0.0046676829125041576, '9224'),
 (0.004662937393665971, '8460'),
 (0.004552416574830116, '39667'),
 (0.004536460467870352, '27779'),
 (0.004386776137097036, '30548'),
 (0.0042996828500316735, '33994'),
 (0.004252860095274651, '7977'),
 (0.004055435191063

# Dropping Lower Ranked Features

In [85]:
# Create a dictionary of features and accuracy scores
features_accuracy = {}
feature_percents = [0.1, 0.2, 0.3, 0.4]
for percent in feature_percents:
    # Define the features and target sets
    important_features = []
    sum = 0
    count = 0
    while sum < percent:
        sum = sum + sorted_features[count][0]
        important_features.append(sorted_features[count][1])
        count+=1
    X = filtered_tf_idf2[important_features]
    y = filtered_tf_idf2['category_id'].ravel()
    y = y.astype('int')
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    # Scale the data
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    # Fit the random forest model
    rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
    t0 = time.time()
    rf_model = rf_model.fit(X_train_scaled, y_train)
    t1 = time.time()
    print(f'Run time to fit model with {percent*100}% of total features: {t1-t0} seconds')
    # Make predictions
    predictions = rf_model.predict(X_test_scaled)
    acc_score = accuracy_score(y_test, predictions)
    features_accuracy[percent] = acc_score

Run time to fit model with 10.0% of total features: 3.247677803039551 seconds
Run time to fit model with 20.0% of total features: 4.390316963195801 seconds
Run time to fit model with 30.0% of total features: 4.797331809997559 seconds
Run time to fit model with 40.0% of total features: 6.618571043014526 seconds


In [86]:
# Print feature percents and accuracy scores
for percent in features_accuracy:
    acc_score = features_accuracy[percent]
    print(f'Accuracy Score for {percent*100}% of total features: {acc_score}')

Accuracy Score for 10.0% of total features: 0.5944206008583691
Accuracy Score for 20.0% of total features: 0.6384120171673819
Accuracy Score for 30.0% of total features: 0.6427038626609443
Accuracy Score for 40.0% of total features: 0.6319742489270386


In [87]:
# Get the highest accuracy score
best_percent = max(features_accuracy, key=features_accuracy.get)
best_acc = features_accuracy[best_percent]
print(f'The highest accuracy score of {best_acc} was achieved using {best_percent*100}% of all features.')

The highest accuracy score of 0.6427038626609443 was achieved using 30.0% of all features.
